In [98]:
model_1 = "gpt-4o"
model_2 = "Qwen-VL-72B-Instruct"

model_1 = model_1.replace("-", "_")
model_2 = model_2.replace("-", "_")

In [99]:
import datasets

data = datasets.load_dataset("lmms-lab/LiveBenchDetailedResults", "2024-09")

In [100]:
model_data_1 = data[model_1].to_pandas()
model_data_2 = data[model_2].to_pandas()

In [101]:
model_data_1.columns

Index(['id', 'images', 'question', 'ground_truth', 'criteria', 'subtask',
       'website', 'response', 'score', 'reason'],
      dtype='object')

In [102]:
same_data = ["images", "question", "ground_truth", "criteria", "subtask", "website"]
model_data_2.drop(columns=same_data, inplace=True)
merged_data = model_data_1.merge(model_data_2, on="id", suffixes=(f"_1", f"_2"))

In [103]:
merged_data.head()

,id,images,question,ground_truth,criteria,subtask,website,response_1,score_1,reason_1,response_2,score_2,reason_2
0,41,[{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIH...,Identify each vulnerability's CVE identifier a...,1. CVE Identifier: CVE-2024-29824\nDescription...,Total score: 10 points.\n\nScoring:\n- For eac...,Concrete Recognition,technology,Here are the vulnerabilities along with their ...,10.00,The assistant response correctly identified al...,1. **CVE-2024-29824**\n - **Description**: I...,10.00,The assistant response correctly identifies ea...
1,274,[{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIH...,List all the news headlines in the order they ...,1. 'I'm amazed I lived to tell the tale': Awar...,"0-1 correct: 0 points, 2-4 correct: 4 points, ...",Concrete Recognition,artandculture,1. **‘I’m amazed I lived to tell the tale’: Aw...,10.00,The assistant response correctly matches all t...,,0.00,No response
2,215,[{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIH...,List all the news headlines related to the 202...,Here are the news headlines related to the 202...,The total score is 10 points. Listing each hea...,Concrete Recognition,sports,Here are the news headlines related to the 202...,4.29,The assistant response correctly identified th...,"1. ""Who will be the No. 1 pick in the 2025 NBA...",8.58,The assistant response correctly identified 6 ...
3,215,[{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIH...,List all the news headlines related to the 202...,Here are the news headlines related to the 202...,The total score is 10 points. Listing each hea...,Concrete Recognition,sports,Here are the news headlines related to the 202...,4.29,The assistant response correctly identified th...,```markdown\n# Interview with Abdoulaye Trésor...,10.00,The assistant's response accurately captures t...
4,18,[{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIH...,Identify and list the three formats provided b...,The three formats provided by the new annotati...,To score the response:\n\n- Award 10 points if...,Concrete Recognition,technology,The new `annotationlib` module in Python 3.14 ...,10.00,The assistant correctly identified and describ...,The three formats provided by the new annotati...,10.00,The assistant correctly identified and describ...


In [104]:
delta = merged_data["score_1"] - merged_data["score_2"]

In [105]:
merged_data["delta"] = delta

In [106]:
sorted_data = merged_data.sort_values("delta", ascending=False)

In [107]:
sorted_data.columns

Index(['id', 'images', 'question', 'ground_truth', 'criteria', 'subtask',
       'website', 'response_1', 'score_1', 'reason_1', 'response_2', 'score_2',
       'reason_2', 'delta'],
      dtype='object')

In [108]:
features = data[model_1].features.copy()

In [109]:
for feature in ["score", "reason", "response"]:
    feat = features.pop(feature)
    features[f"{feature}_1"] = feat
    features[f"{feature}_2"] = feat

In [110]:
features["delta"] = datasets.Value("float32")

In [111]:
features

{'id': Value(dtype='int32', id=None),
 'images': Sequence(feature=Image(mode=None, decode=True, id=None), length=-1, id=None),
 'question': Value(dtype='string', id=None),
 'ground_truth': Value(dtype='string', id=None),
 'criteria': Value(dtype='string', id=None),
 'subtask': Value(dtype='string', id=None),
 'website': Value(dtype='string', id=None),
 'score_1': Value(dtype='float32', id=None),
 'score_2': Value(dtype='float32', id=None),
 'reason_1': Value(dtype='string', id=None),
 'reason_2': Value(dtype='string', id=None),
 'response_1': Value(dtype='string', id=None),
 'response_2': Value(dtype='string', id=None),
 'delta': Value(dtype='float32', id=None)}

In [112]:
def gen():
    for i, row in sorted_data.iterrows():
        yield row.to_dict()


final_data = datasets.Dataset.from_generator(gen, features=features)

Generating train split: 218 examples [00:00, 329.76 examples/s]


In [113]:
final_data.push_to_hub("lmms-lab/LiveBenchDetailedResultsComparison", "2024-09", split=f"{model_1}_vs_{model_2}")

Uploading the dataset shards: 100%|██████████| 1/1 [00:14<00:00, 14.44s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/lmms-lab/LiveBenchDetailedResultsComparison/commit/f0941a0afeb32f8f0aae1861e6ef658a237bb249', commit_message='Upload dataset', commit_description='', oid='f0941a0afeb32f8f0aae1861e6ef658a237bb249', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/lmms-lab/LiveBenchDetailedResultsComparison', endpoint='https://huggingface.co', repo_type='dataset', repo_id='lmms-lab/LiveBenchDetailedResultsComparison'), pr_revision=None, pr_num=None)